| Image 1 | Image 2 |
|--------|---------|
| <img src="101_4.png" width="300"/> | <img src="Screenshot 2025-11-18 204604.png" width="500"/> |


In [1]:
from transformers import AutoImageProcessor, ResNetForImageClassification
import torch
from PIL import Image
import numpy as np

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
model.eval()


c:\agentic ai\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\agentic ai\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mukun\.cache\huggingface\hub\models--microsoft--resnet-50. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https:

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [12]:
import torch.nn.functional as F

def get_fp_embedding(image_path):
    img = Image.open(image_path).convert("RGB")
    inputs = processor(images=img, return_tensors="pt")

    with torch.no_grad():
        # get last hidden state from model backbone
        outputs = model(**inputs, output_hidden_states=True)

        # outputs.hidden_states is a list of feature maps
        # ResNet-50 last block feature map = (1, 2048, 7, 7)

        feature_map = outputs.hidden_states[-1]       # shape: (1, 2048, 7, 7)
        emb = torch.mean(feature_map, dim=[2, 3])     # global average pooling → (1, 2048)
        emb = F.normalize(emb, p=2, dim=1)            # normalize

    return emb.squeeze().numpy()                      # final shape = (2048,)


In [13]:
import os

root = "dataset"      # <<< change this to your folder path
database = {}

for person in os.listdir(root):
    person_path = os.path.join(root, person)
    if not os.path.isdir(person_path):
        continue

    print("Processing Person:", person)

    embeddings = []
    for img_file in os.listdir(person_path):
        img_path = os.path.join(person_path, img_file)
        emb = get_fp_embedding(img_path)
        embeddings.append(emb)

    # Average embedding for that person (10 images → 1 vector)
    database[person] = np.mean(embeddings, axis=0)


Processing Person: subject 10
Processing Person: subject 4
Processing Person: subject 6
Processing Person: subject 7
Processing Person: subject 8
Processing Person: subject 9
Processing Person: subject1
Processing Person: subject2
Processing Person: subject3
Processing Person: subject5


In [14]:
test_img = list(database.values())[0]   # take first embedding
print("Embedding shape =", test_img.shape)


Embedding shape = (2048,)


In [15]:
import faiss

dim = 2048
index = faiss.IndexFlatL2(dim)



In [16]:
ids = list(database.keys())
matrix = np.vstack([database[i] for i in ids]).astype("float32")

index.add(matrix)

In [17]:
def identify_fingerprint(query_img):
    q = get_fp_embedding(query_img).astype("float32")

    # Search top-1 match
    D, I = index.search(np.array([q]), 1)

    matched_person = ids[I[0][0]]
    distance = D[0][0]

    # Convert L2 distance → similarity score
    score = 1 / (1 + distance)

    return matched_person, score


In [32]:
query = "107_2.tif"   # place test fingerprint here

person, score = identify_fingerprint(query)

print("Predicted Person:", person)
print("Match Score:", score)


Predicted Person: subject 9
Match Score: 0.69838053
